In [1]:
import cv2
import numpy as np
import os
import tensorflow as tf

In [26]:
cwd = os.getcwd()
img_path = cwd + "\\sample_input_image.jpg"
img_path2 = cwd + "\\aww.jpg"
vid_path = cwd + "\\pedestrian_survaillance.mp4"
vid_path2 = cwd + "\\Blick.mp4"
classes = ["None", 'Kinder Country', 'Knoppers', 'PickUp Blue', 'PickUp Brown', 'StudentFutter', 'Twix']
img = cv2.imread(img_path)
img2 = cv2.imread(img_path2)

In [3]:
model = tf.saved_model.load("Faster_Blick4")
model = model.signatures['serving_default']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [19]:


def draw_bounding_boxes(model, image, clas):

    imagenp = image
    width = imagenp.shape[1]
    height = imagenp.shape[0]
    imagenp = np.expand_dims(imagenp, axis=0)
    imagenp = tf.convert_to_tensor(imagenp)
    pred = model(imagenp)
    
#     print(pred)
#     cv2.line(image, (300, 0), (300, height), color=(0, 255, 0), thickness=3)
    for x in range(pred['detection_boxes'][0].shape[0]):
        det_score = pred['detection_scores'][0][x].numpy()
        
        if det_score <= 0.0:
            g = image
            continue
        elif det_score >= 0.85:
#             print(pred['detection_classes'][0][x].numpy())
#             print(pred['detection_scores'][0][x])
            ymin,xmin,ymax,xmax=pred['detection_boxes'][0][x].numpy()

            (left, right, top, bottom) = (xmin * width, xmax * width, ymin * height, ymax * height)
            left = int(left)
            right = int(right)
            top = int(top)
            bottom = int(bottom)
            crop = image[top:bottom, left:right]

    #         gray_image = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)

    #         # convert the grayscale image to binary image
    #         _,thresh = cv2.threshold(gray_image,127,255,0)

    #         # calculate moments of binary image
    #         M = cv2.moments(thresh)
#             cl = label(int(pred['detection_classes'][0][x].numpy()))
            cl = int(pred['detection_classes'][0][x].numpy())
            print(cl)
            print(clas[cl])
#             print(int(pred['detection_classes'][0][x].numpy()))


    #         # calculate x,y coordinate of center
    #         cX = int(M["m10"] / M["m00"])
    #         cY = int(M["m01"] / M["m00"])

    # #         if cX == 300:
    # #             print("aw")
    #         # put text and highlight the center
            cx = int(round((right - left)/2)) + left
            cy = int(round((bottom - top)/2)) + top
    #         print(cx)#, cy)
    #         if framecount % 15 == 0:
    #             frames.insert(0, (cx, cy))
    #             print(frames)
    #             print(frames[0][0], frames[0][1])
    #             if len(frames) >= 3:
    #                 frames.pop(-1)
    # #                 print(frames)
    #             cv2.line(image, (frames[0][0], frames[0][1]), (cx, cy), color=(0, 255, 0), thickness=3)
    #         if cx == 300:
    #             counter = 1
    #             cv2.line(image, (300, 0), (300, height), color=(0, 0, 255), thickness=3)
    #         cv2.circle(image, (cx, cy), 5, (0, 255, 0), -1)
    #         cv2.putText(image, "{}, {}, ID: {}".format(str(cx), str(cy), x), (cx - 25, cy - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    #         cv2.circle(image, (left, top), 5, (0, 255, 0), -1)
    #         cv2.circle(image, (right, bottom), 5, (0, 255, 0), -1)
    #         cv2.putText(img, "{}, {}".format(left, top), (left - 25, top - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    #         cv2.putText(img, "{}, {}".format(right, bottom), (right - 25, bottom - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    #         cv2.putText(img, "centroid", (cX + (right - left) - 25, cY  + (bottom - top) - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            cv2.putText(image, "{}, {}%".format(clas[cl], int(round(float(det_score), 2)*100)), 
                        (cx - 50 , cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            g = cv2.rectangle(image, (left, top), (right, bottom), color=(0, 255, 0), thickness=1)
            
#         break
    return g, crop



In [25]:
cv = cv2.VideoCapture(vid_path2)
count = 0
cs = []
framecount = 0
W = int(cv.get(3))
H = int(cv.get(4))
print(W, H)
fourcc = cv2.VideoWriter_fourcc(*"XVID")
writer = cv2.VideoWriter("output.avi", fourcc, 30, (W, H), True)

while cv.isOpened():
    ret, frame = cv.read()
    framecount += 1
#     print(framecount)
    if ret and (framecount >= 180):
        f, crop = draw_bounding_boxes(model, frame, classes)
#         count = count + counter
        writer.write(f)
        cv2.imshow("Reading", f)
        

        if cv2.waitKey(1) == ord('q'):
            break
    elif ret and (framecount < 180):
        writer.write(frame)
        cv2.imshow("Reading", frame)
        if cv2.waitKey(1) == ord('q'):
            break
#         if cv2.waitKey(30) == ord('q'):
#             break
#         continue
    else:
        break
print("Final Count: ", framecount)
cv.release()
writer.release()
cv2. destroyAllWindows() 

640 352
2
Knoppers
2
Knoppers
2
Knoppers
2
Knoppers
2
Knoppers
2
Knoppers
1
Kinder Country
2
Knoppers
1
Kinder Country
Final Count:  186


In [28]:
f, crop= draw_bounding_boxes(model, img2, classes)

# for x in range(len(crop_img) + 1):
try:
    cv2.imshow("x", f)
    cv2.waitKey(10000)
except:
    pass

2
Knoppers
